In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers

current_learning_iteration = 0

# Device Class
class Device:
    def __init__(self, device_id, ram, storage, cpu, bandwidth, battery, charging):
        self.device_id = device_id
        self.ram = ram
        self.storage = storage
        self.cpu = cpu
        self.bandwidth = bandwidth
        self.battery = battery
        self.charging = charging
        self.energy_consumption = ram + storage + cpu + bandwidth
        self.model = self.create_model()
        self.last_round_participated = 0
        self.data = None  # Placeholder for dataset partition

        self.test_data = None
        # TODO: add test data for each device
        # TODO: use the test data retrieved from load dataset from mnist and distribute it between devices and global model


        self.number_of_times_fitted = 0

    def create_model(self):
        model = keras.Sequential([
            layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                      # new
                      loss='categorical_crossentropy', metrics=['accuracy'])
        return model




# Global Model
# Define the global model with the same architecture
def create_global_model():
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                  # new
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

global_model = create_global_model()


def multiply_tensors(a, b):
    if isinstance(a, (int, float)) and isinstance(b, (int, float)):
        return a * b
    elif isinstance(a, list) and isinstance(b, list):
        return [multiply_tensors(x, y) for x, y in zip(a, b)]
    else:
        raise ValueError("Incompatible types for multiplication")

def add_tensors(a, b):
    if isinstance(a, (int, float)) and isinstance(b, (int, float)):
        return a + b
    elif isinstance(a, list) and isinstance(b, list):
        return [add_tensors(x, y) for x, y in zip(a, b)]
    else:
        raise ValueError("Incompatible types for addition")

def sum_all_nested_lists(list_of_lists):
    def recursive_sum(lists):
        if isinstance(lists[0], list):
            return [recursive_sum([lst[i] for lst in lists]) for i in range(len(lists[0]))]
        else:
            return sum(lists)
    
    return recursive_sum(list_of_lists)

def multiply_nested_list(lst, factor):
    result = []
    for item in lst:
        if isinstance(item, list):
            # Recursively handle sublists
            result.append(multiply_nested_list(item, factor))
        else:
            # Multiply number
            result.append(item * factor)
    return result

import numpy as np
def aggregate_weights(devices):
    """Computes the weighted average of model weights from all devices and updates the global model."""

    num_devices = len(devices)
    if num_devices == 0:
        print("No devices available for aggregation.")
        return

    device_weights_all_layers = []
    device_participation_ratio = []
    data_lengths = []

    for device in devices:
        device_weights_all_layers.append(device.model.get_weights())
        print("*******************")
        print(device.device_id)
        print("this device's weights:")
        print(device.model.get_weights())
        
        device_participation_ratio.append(device.last_round_participated / current_learning_iteration)
        print("this device's participation ratio:")
        print(device.last_round_participated / current_learning_iteration)
        
        data_lengths.append(len(device.data[0]))
        print("this device's data to all ratio:")
        print(len(device.data[0])/60000.0)

    # device_participation_ratio = np.array(device_participation_ratio)
    # data_lengths = np.array(data_lengths)
    # data_fractions = data_lengths / data_lengths.sum()
    sum_data = 0
    for data_len in data_lengths:
        sum_data += data_len
    
    data_fractions = []
    for device in devices:
        data_fractions.append(len(device.data[0])/float(sum_data))
    
    
    # device_final_weights = device_participation_ratio * data_fractions  # shape: (num_devices,)
    # for item in device_final_weights:
    #     print(item)
    

    # Transpose the weights to group by layer
    # num_layers = len(device_weights_all_layers[0])

    aggregated_weights_devices = []
    for d in range(len(devices)):
        aggregated_weights_devices.append(multiply_nested_list(devices[d].model.get_weights(), data_fractions[d]*device_participation_ratio[d]))
    
    aggregated_weights = sum_all_nested_lists(aggregated_weights_devices)
    # TODO: Weighted multiplication for each node in each layer of the neural network of the received devices and then summing
    #       the related parts together so that we get a full weighted average of all these devices' models
    
    # for layer_idx in range(num_layers):
    #     print("layer no:",layer_idx)
    #     for item in devices[0].model.get_weights():
    #         print("layer-----------------")
    #         print(item)
    #         print(len(item))
    #         print("-----------------")
    #         for item2 in item:
    #             print(item2)
    #             print("=========================")
    #             for item3 in item2:
    #                 print(item3)
    #                 print("++++++++++++++++++++++++++++++++++++++")
    #                 for item4 in item3:
    #                     print(item4)
    # multiply_nested_list()
    #     aggregated_weights.append(weighted_layer)


    print("Aggregated weights:")
    for layer_idx, layer_weights in enumerate(aggregated_weights):
        print(f"Layer {layer_idx}: {layer_weights.shape}")
    return aggregated_weights


KeyboardInterrupt: 

In [26]:
# Load dataset from CSV
csv_file = 'devices.csv'
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip().str.lower()


# Convert CSV rows into device objects
devices = []

for _, row in df.iterrows():
    device = Device(
        row['id'], row['ram'], row['storage'], row['cpu'], row['bandwidth'], row['battery'],
        row.get('charging', 0)
    )
    devices.append(device)


# Load MNIST dataset  # new
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()



# new
from keras.utils import to_categorical
# Convert labels to categorical (one-hot encoded)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Normalize data and reshape for CNN
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)  # Add channel dimension

# new
x_test = x_test.astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1)  # Add channel dimension

# new
# Shuffle data
# indices = np.arange(len(x_train))
# np.random.shuffle(indices)
# x_train, y_train = x_train[indices], y_train[indices]

# new
# Correct test split
split_index = int(0.8 * len(x_test))
x_test_devices, y_test_devices = x_test[:split_index], y_test[:split_index]
x_test_global, y_test_global = x_test[split_index:], y_test[split_index:]

# Training data (for devices)
x_train_devices, y_train_devices = x_train, y_train

# Split training data among devices
num_devices = len(devices)
split_size = len(x_train_devices) // num_devices

for i, device in enumerate(devices):
    start = i * split_size
    end = (i + 1) * split_size if i < num_devices - 1 else len(x_train_devices)
    device.data = (x_train_devices[start:end], y_train_devices[start:end])

# Split test data (device-level)
split_size = len(x_test_devices) // num_devices

for i, device in enumerate(devices):
    start = i * split_size
    end = (i + 1) * split_size if i < num_devices - 1 else len(x_test_devices)
    device.test_data = (x_test_devices[start:end], y_test_devices[start:end])


# First Iteration
bitstring = [1 for _ in range(len(devices))]
print(bitstring)

LAST_PARTICIPATED_ROUND_FOR_ALL_DEVICES = [1 for _ in range(len(devices))]
print(len(LAST_PARTICIPATED_ROUND_FOR_ALL_DEVICES))
LAST_WEIGHTS_SENT_FOR_ALL_DEVICES = [None for _ in range(len(devices))]

def fit_bitstring_devices(bitstring):
    global current_learning_iteration
    global LAST_PARTICIPATED_ROUND_FOR_ALL_DEVICES
    global LAST_WEIGHTS_SENT_FOR_ALL_DEVICES
    current_learning_iteration += 1
    for device in devices[:2]:
        print(int(device.device_id))
        if bitstring[int(device.device_id)] == 1:
            device.model.fit(device.data[0], device.data[1], epochs=7, verbose=1)
            print(device.device_id)
            print(len(LAST_PARTICIPATED_ROUND_FOR_ALL_DEVICES))
            LAST_PARTICIPATED_ROUND_FOR_ALL_DEVICES[int(device.device_id)] = current_learning_iteration
            LAST_WEIGHTS_SENT_FOR_ALL_DEVICES[int(device.device_id)] = device.model.get_weights()

            device.number_of_times_fitted += 1
            device.last_round_participated = current_learning_iteration

def give_global_model_weights_to_bitstring_devices(bitstring):
    for device in devices:
        if int(bitstring[int(device.device_id)]) == 1:
            device.model.set_weights(global_model.get_weights())
# Call this function after training the local models:
# aggregate_weights(global_model, devices)

fit_bitstring_devices(bitstring)

# new
# print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(devices))
give_global_model_weights_to_bitstring_devices(bitstring)
# print(global_model.get_weights())
print("------------------------------------------------------------")


test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

d:\github project\FL\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
100
0
Epoch 1/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0746 - loss: 2.3125
Epoch 2/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1599 - loss: 2.2806
Epoch 3/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3601 - loss: 2.2383
Epoch 4/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5303 - loss: 2.1928
Epoch 5/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6492 - loss: 2.1122
Epoch 6/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6866 - loss: 2.0146
Epoch 7/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7432 - loss: 1.8448
0.0
100
1
Epoch 1/7
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.1527 - loss: 2.2887
Epoch 2/7
1

ValueError: You called `set_weights(weights)` on layer 'sequential_1410' with a weight list of length 0, but the layer was expecting 6 weights.

In [7]:
!pip install pymoo

In [ ]:
import numpy as np
import random
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.termination.default import DefaultMultiObjectiveTermination
from keras.models import clone_model


# Parameters
NUM_DEVICES = num_devices   # Number of devices (length of bitstring)
POPULATION_SIZE = 50
NUM_GENERATIONS = 3

# Step 1: Define the Problem
import numpy as np
from pymoo.core.problem import Problem

class FederatedLearningProblem(Problem):
    def __init__(self, num_devices, devices, global_model, x_test_global, y_test_global):
        super().__init__(
            n_var=num_devices,         # Number of variables (bitstring length)
            n_obj=3,                   # Number of objectives
            n_constr=0,                # No constraints
            xl=np.zeros(num_devices),  # Lower bound (0)
            xu=np.ones(num_devices),   # Upper bound (1)
            type_var=np.bool_          # Binary variables (bitstrings)
        )
        self.devices = devices
        self.global_model = global_model
        self.x_test_global = x_test_global
        self.y_test_global = y_test_global

        # Save the initial global model weights
        self.initial_global_weights = global_model.get_weights()

    def _evaluate(self, X, out, *args, **kwargs):
        """Evaluates objective values for each solution in the population."""
        num_solutions = len(X)
        F = np.zeros((num_solutions, 3))  # Initialize objective matrix

        for i, bitstring in enumerate(X):
            # Reset the global model to its initial state
            # Update device participation based on the bitstring
            selected_devices = [device for device, bit in zip(self.devices, bitstring) if int(bit) == 1]
            # Objective 1: Hardware Objectives (maximize)

            # new
            hardware_score = 0.0
            for device in selected_devices:
                device_hardware_score = float(6 - (device.ram + device.storage + device.cpu + device.bandwidth + device.battery + device.charging)) / 6.0
                hardware_score += device_hardware_score

            F[i, 0] = hardware_score  # Minimize (negative of hardware score)

            # new
            fairness_score = 0
            for device in self.devices:
                if bitstring[int(device.device_id)] == 1:
                    # new
                    _, accuracy = global_model.evaluate(device.test_data[0], device.test_data[1], verbose=0)
                    fairness_score += accuracy

            F[i, 1] = fairness_score/float(len(selected_devices))  # Minimize (negative of fairness score)  # Added (/Selected Devices) to normalize between 0 and 1
            
            # Objective 3: Global Model Accuracy (maximize)
            temp_global_model = clone_model(global_model)
            temp_global_model.set_weights(aggregate_weights(selected_devices))
            _, global_accuracy = temp_global_model.evaluate(self.x_test_global, self.y_test_global, verbose=0)
            F[i, 2] = 1 - global_accuracy  # Minimize (1 - accuracy)
        out["F"] = F  # Set the objective values


In [9]:


problem = FederatedLearningProblem(
    num_devices=NUM_DEVICES,
    devices=devices,
    global_model=global_model,
    x_test_global=x_test_global,
    y_test_global=y_test_global
)


# Step 2: Configure NSGA-II Algorithm
algorithm = NSGA2(
    pop_size=POPULATION_SIZE,
    sampling=BinaryRandomSampling(),      # Random bitstrings
    crossover=TwoPointCrossover(),        # Two-point crossover
    mutation=BitflipMutation(),           # Bit flip mutation
    eliminate_duplicates=True             # Avoid duplicate solutions
)


NUMBER_OF_LEARNING_ITERATIONS = 2

In [10]:
# DEBUG:
print(x_test_global.shape)
print(y_test_global.shape)

print(x_train.shape)
print(y_train.shape)

(2000, 28, 28, 1)
(2000, 10)
(60000, 28, 28, 1)
(60000, 10)


In [ ]:
# Step 3: Run Optimization
print("GLOBAL MODEL BEFORE OPTIMIZATION")
print(global_model.get_weights())

res = minimize(
    problem=problem,
    algorithm=algorithm,
    # termination=MultiObjectiveSpaceToleranceTermination(tol=1e-6, n_last=10, nth_gen=5, n_max_gen=NUM_GENERATIONS),
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)
print("GLOBAL MODEL AFTER OPTIMIZATION")
print(global_model.get_weights())

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      2 |             - |             -
     2 |      100 |      4 |  0.3068293454 |         ideal
     3 |      150 |      9 |  0.2503217633 |         ideal


In [ ]:
# Step 4: Extract the Best Pareto Front
pareto_front = res.F   # Objective values of solutions in Pareto front
pareto_solutions = res.X  # Corresponding bitstrings

# Print the Best Pareto Front Solutions
print("Best Pareto Front (Bitstrings):")
for bitstring in pareto_solutions:
    print("".join(map(str, bitstring)).replace('True','1').replace('False','0'))

bitstring = pareto_solutions[0] # for now!
bitstring = str(bitstring).replace('False','0').replace('True','1')
for char in bitstring:
    if char != '0' and char != '1':
        bitstring = bitstring.replace(char,'')

print(len(bitstring))
print(bitstring)
temp_bitstring = []
for bit in bitstring:
    temp_bitstring.append(bit)
bitstring = temp_bitstring

########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# Aggregate weights to update the global model
# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(devices))
print(global_model.get_weights())
print("------------------------------------------------------------")

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.model.fit(device.data[0], device.data[1], epochs=5, verbose=0)
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration


test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

Best Pareto Front (Bitstrings):
0000111100110100000000000010000101001000001110110011111100100110100010011001010001001010110100100010
0000111100110110000000000010001100101011000001101000101000100110100010011001010001001010110100100010
1011010111000011000010100110000000100101010001100100000001000100010110010011001001010110100000001100
0101010010010000001111100011010011001000001110110010111011100111010010000010001010111101101011000100
0110100110001010001110000100001000100011101110101010101111000110111100100111000010000110110011100100
0101010010010010001111100011010011001111000011011101101100101110100010010010001010111101101011100100
0001000000100100000000000011100101000101110100011101111010011100101100000101001001100000001011010010
0100000100011100011100011110010000000001001011001110111100100110100010011011000000010101100101000110
1011011100001110010010111000001000000000010101100100000001000100010110000011001001010110100000001100
100
0000111100110100000000000010000101001000001110110011111

In [13]:
print(bitstring) # for now!
print(len(bitstring))
print(bitstring.count("1"))

['0', '0', '0', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0']
100
39


In [14]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      5 |             - |             -
     2 |      100 |      6 |  0.0129361668 |         ideal
     3 |      150 |      5 |  0.0786670050 |         ideal


In [ ]:
print(awiodawo)

In [15]:
########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")

w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

Device 4.0: Loss before: [2.2980031967163086, 0.14666666090488434], Loss after: [2.284519910812378, 0.35333332419395447]
Device 5.0: Loss before: [2.298715114593506, 0.11999999731779099], Loss after: [2.2860357761383057, 0.34333333373069763]
Device 6.0: Loss before: [2.298321008682251, 0.14166666567325592], Loss after: [2.2865428924560547, 0.4116666615009308]
Device 7.0: Loss before: [2.298210859298706, 0.13833333551883698], Loss after: [2.282723903656006, 0.22833333909511566]
Device 10.0: Loss before: [2.2980704307556152, 0.1616666615009308], Loss after: [2.2842841148376465, 0.2266666740179062]
Device 11.0: Loss before: [2.2986857891082764, 0.13333334028720856], Loss after: [2.285165786743164, 0.22333332896232605]
Device 13.0: Loss before: [2.2982802391052246, 0.14499999582767487], Loss after: [2.2767813205718994, 0.1550000011920929]
Device 26.0: Loss before: [2.2987937927246094, 0.12166666984558105], Loss after: [2.286884307861328, 0.3266666531562805]
Device 31.0: Loss before: [2.298

In [16]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")


w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      4 |             - |             -
     2 |      100 |      6 |  0.2727559400 |         ideal
     3 |      150 |      9 |  0.0710064480 |         ideal
Device 4.0: Loss before: [2.299807071685791, 0.12333333492279053], Loss after: [2.2950029373168945, 0.12333333492279053]
Device 5.0: Loss before: [2.300266981124878, 0.10833333432674408], Loss after: [2.295975685119629, 0.20333333313465118]
Device 6.0: Loss before: [2.3000829219818115, 0.1133333370089531], Loss after: [2.296647548675537, 0.1133333370089531]
Device 7.0: Loss before: [2.299997329711914, 0.10333333164453506], Loss after: [2.294761896133423, 0.13500000536441803]
Device 10.0: Loss before: [2.2999212741851807, 0.11666666716337204], Loss after: [2.2957682609558105, 0.15000000596046448]
Device 11.0: Loss before: [2.3003227710723877, 0.1133333370089531], Loss after: [2.296252965927124, 0.2266666740179062]
Device 13.0: Loss before: [2.299876928329

In [17]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(global_model, selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")


w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      5 |             - |             -
     2 |      100 |      6 |  0.0564924725 |         ideal
     3 |      150 |      8 |  0.4483626535 |         ideal
Device 4.0: Loss before: [2.3013622760772705, 0.12333333492279053], Loss after: [2.298837661743164, 0.12333333492279053]
Device 5.0: Loss before: [2.3016862869262695, 0.10833333432674408], Loss after: [2.2995495796203613, 0.10833333432674408]
Device 6.0: Loss before: [2.3016045093536377, 0.1133333370089531], Loss after: [2.300372362136841, 0.1133333370089531]
Device 7.0: Loss before: [2.3015823364257812, 0.10333333164453506], Loss after: [2.2990756034851074, 0.12166666984558105]
Device 10.0: Loss before: [2.3015029430389404, 0.11666666716337204], Loss after: [2.299830198287964, 0.11666666716337204]
Device 11.0: Loss before: [2.3018038272857666, 0.1133333370089531], Loss after: [2.300283908843994, 0.1133333370089531]
Device 13.0: Loss before: [2.301440000

TypeError: aggregate_weights() takes 1 positional argument but 2 were given

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.device_id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(global_model, devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.device_id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")


w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")